# カタカナ5文字を自動識別するモデルの構築 - 最終発表
## 中間発表1回目まとめ
* データセット読み込み
 - アイウエオ各200枚の画像をベクトル形式に読み込み訓練データセット、検証データセット、テストデータセットに分割


* ニューラルネットワークのビルディングブロックとなるレイヤの定義
 - Affineレイヤ、ReLUレイヤ、Softmaxレイヤ、BatchNormalizationレイヤ
 - SGDレイヤ、RMSPropレイヤ
 - 多層MLPの定義・検証
 - 隠れ層が2層と3層の場合の比較
 - SGDとRMSPropの比較
 - エポック数調整
 - ミニバッチサイズ調整

### 課題識別精度

8/19時点

    Test loss:0.12792950478432882 
    Test accuracy:0.9707692307692307

### 指摘事項

* 純粋に正則化の効果を狙う場合、重み減衰やドロップアウトが一般的
* 学習率を下げてみよう
* 交差検証によって検証データ数を増やしてハイパーパラメータを決るとよい
* 今回の3層ネットワークのように過学習になる場合は正則化しよう
* エポック数の検証ははじめに長く計算し結果を評価、途中できれば十分 or 早期終了

## 中間発表2回目まとめ
- CNN実装
 - Convolutionレイヤ、MaxPoolingレイヤ
- データ拡張
- 交差検証
 - レイヤ構成、オプティマイザ（SGD/RMSProp, 学習率）、バッチサイズ
- 課題識別精度
- 最終発表に向けて

### 課題識別精度

9/1時点

    Test loss:0.07289509119221133 
    Test accuracy:0.9866153846153846 
    
## 最終発表
- 実装の進捗状況
- モデルの改良点と識別精度の変遷
- arxiv.org等で見つけた論文とそこから得た知見
- 課題を通しての感想

### 実装の進捗状況
- Global Average Pooling を実装
 - 訓練データをファイルから都度読む対応と合わせ、層を深くすることができた
- Weight Decay と Adam, Momentum を実装
 - 汎化性能向上を見込んでWeight Decay
 - Weight Decay時にAdamは向かなかったのでMomentumを使用
  - ドロップアウトを試しても良かったかも？

In [1]:
class GlobalAveragePooling:
    def __init__(self):
        self.x = None

    def forward(self, x, train_flg):
        out = x.mean(axis=(2, 3))
        self.x = x
        return out

    def backward(self, dout, weight_decay_lambda):
        n, c, h, w = self.x.shape
        area = h * w
        dx = dout.repeat(area).reshape(n, c, h, w)
        dx /= area
        return dx

### モデルの改良点と識別精度の変遷
- 中間発表１
 - 全層結合型ニューラルネットワーク
  - Test accuracy:0.9707692307692307

        Affine(784, 50) -> BatchNoralization -> ReLU -> Affine(50, 5) -> SoftmaxWithLoss

- 中間発表２
 - 畳み込みニューラルネットワーク, データ拡張(+1200)
 - Test accuracy:0.9866153846153846 

        Convolution(3x3x16, stride:1) -> BatchNormalization -> ReLU -> MaxPooling(4x4, stride:4) ->
        Convolution(3x3x32, stride:1) -> BatchNormalization -> ReLU -> MaxPooling(2x2, stride:2) ->
        Affine(512, 5) -> SoftmaxWithLoss
        
- 最終発表
 - 中間発表２ + データ拡張(+5000), Global Average Pooling, Weight Decay
 - Test accuracy:0.9978461538461538

         Convolution(5x5x16, stride:1) -> BatchNormalization -> ReLU -> MaxPooling(2x2, stride:4) ->
         Convolution(3x3x64, stride:1) -> BatchNormalization -> ReLU ->
         Convolution(3x3x64, stride:1) -> BatchNormalization -> ReLU -> MaxPooling(2x2, stride:2) ->
         GlobalAveragePooling -> BatchNormalization -> ReLU -> Affine(64, 5) -> SoftmaxWithLoss 

### arxiv.org等で見つけた論文とそこから得た知見
- Global Average Pooling
 - https://arxiv.org/abs/1312.4400
 - パラメータ数を劇的に減らすことができる
 - 最後のモデルでGlobal Average Poolingを用いなかったら Memory Error になる
- VGG風 3x3 の畳み込みを２つつなげて 5x5 と同じ範囲をカバーしつつ表現力を上げる
 - https://arxiv.org/abs/1409.1556
- Weight DecayでAdamを使う場合は工夫が必要
 - https://arxiv.org/abs/1711.05101
 - Adamの改良まではできなかった
 - SGD, Momentumは普通に使われるらしい
  - https://stats.stackexchange.com/questions/70101/neural-networks-weight-change-momentum-and-weight-decay


### 課題を通しての感想

- ボーダーラインは超えることができたが、シンプルなカタカナ５文字でも99%を超えてからなかなか精度が上がらず学習の難しさを痛感した。
- 計算機のメモリが少なめ？でパラメータ数削減の効力を身にして感じた。
- データ拡張をどこまで行えばよいのか見当がつかなかった。適当に拡張してもモデルがそれにフィットしすぎて精度が上がらない現象（過学習）に遭遇。
 - 訓練したモデルの最良のエポック数は精度が一番良く（誤差が一番低く）なったエポックを採用しても過学習している可能性をはらまないか？検証集合を用いて評価していれば問題ないか？
 - 検証集合に対する精度が１に達してしまった場合、（正則化をかけた上で）誤差をひたすら小さくする方向でチューニングして大丈夫か？
- 中盤はまともに交差検証を行っていたが、終盤は交差検証を行うにも半日以上かかるようになり施行回数が激減してしまった。GPUが必須と言われる理由がよくわかった。